# 请求重试

本笔记介绍 HTTP 请求中常见的重试策略与实现方法，重点给出 Python 中使用 requests + urllib3 的实践示例，并提供一个简单的通用重试装饰器示例与使用 tenacity 的可选方案。

## 目录
1. 为什么需要重试
2. 设计重试策略的要点
3. 使用 requests + urllib3 Retry（推荐）
4. 指数退避与抖动（jitter）
5. 自定义重试装饰器（轻量实现）
6. 使用 tenacity（更强大、可配置）
7. 常见陷阱与建议
8. 小结与参考链接

## 1. 为什么需要重试
- 网络不稳定、短暂的服务端错误（500/502/503/504）、限流（429）等场景经常是瞬时的，重试能显著提高成功率。
- 在爬虫或批量请求中，合理的重试策略能减少失败率并提高鲁棒性。
- 注意：重试并不总是合适，尤其是在非幂等的写操作（POST/PUT/DELETE）上要谨慎。

## 2. 设计重试策略的要点
- 重试次数（max attempts）：避免无限重试，设置合理上限，例如 3-10 次。
- 重试的响应码集合：通常包括 429（过多请求）、5xx（服务器错误），也可根据业务扩展。
- 是否对幂等方法才重试：GET/HEAD/OPTIONS/PUT/DELETE 通常可重试，POST 要谨慎。
- 退避策略（backoff）：固定间隔 / 指数退避 / 指数退避 + 抖动（jitter）。
- 超时设置：每次请求应设置合理的 timeout，防止长时间挂起。
- 监控与日志：记录失败原因与重试次数，避免对目标服务器造成额外压力。

## 3. 使用 requests + urllib3 Retry（推荐）
requests 基于 urllib3，urllib3 提供了一个健壮的 Retry 工具，可以配合 requests 的 HTTPAdapter 使用，示例如下：

In [ ]:
from requests import Session
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

def get_session_with_retries(
    total=5,
    backoff_factor=0.5,
    status_forcelist=(429, 500, 502, 503, 504),
    allowed_methods=("HEAD", "GET", "OPTIONS"),
    raise_on_status=False,  # urllib3 2.x 的行为参数，requests 会处理响应
):
    """返回一个配置好重试策略的 requests.Session
    参数说明：
    - total: 最大重试次数（包含重试触发前的第一次请求）
    - backoff_factor: 退避系数（exponential backoff 公式为 backoff_factor * (2 ** (retry - 1)))
    - status_forcelist: 遇到这些状态码时触发重试
    - allowed_methods: 哪些 HTTP 方法允许自动重试（默认对幂等方法重试）
    """
    session = Session()
    retries = Retry(
        total=total,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
        allowed_methods=allowed_methods,
        raise_on_status=raise_on_status,
    )
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('https://', adapter)
    session.mount('http://', adapter)
    return session

# 简单使用示例（演示对 500 返回的重试行为）：
if __name__ == '__main__':
    s = get_session_with_retries(total=3, backoff_factor=0.2)
    try:
        # httpbin 的 /status/500 会返回 500，此请求将按照重试策略自动重试
        r = s.get('https://httpbin.org/status/500', timeout=5)
        print('最终状态码：', r.status_code)
    except Exception as e:
        print('请求出错：', e)


## 4. 指数退避与抖动（jitter）
- 指数退避（exponential backoff）：每次重试等待时间按指数增长，常见公式 base * (2 ** n)。
- 抖动（jitter）：在指数退避基础上加入随机扰动，避免大量客户端在相同节奏同时重试导致"群体效应"（thundering herd）。
- 简单实现示例见下（用于自定义装饰器场景）。

In [ ]:
import time
import random
import functools
from typing import Callable, Iterable, Tuple

def retry(
    exceptions: Tuple = (Exception,),
    total: int = 3,
    backoff_factor: float = 0.5,
    jitter: float = 0.1,
    allowed_result: Callable[[object], bool] = None,
):
    """一个轻量级的重试装饰器示例
    - exceptions: 捕获哪些异常后触发重试
    - total: 最大重试次数（包括第一次）
    - backoff_factor: 指数退避基数
    - jitter: 在等待时间上的最大随机扰动（以秒为单位）
    - allowed_result: 可选的结果校验函数，返回 False 可触发重试（例如检测返回的 status_code）
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            attempt = 0
            while True:
                attempt += 1
                try:
                    result = func(*args, **kwargs)
                    if allowed_result is not None and not allowed_result(result):
                        raise RuntimeError('结果不满足 allowed_result 条件')
                    return result
                except exceptions as e:
                    if attempt >= total:
                        raise
                    # 指数退避 + 抖动
                    backoff = backoff_factor * (2 ** (attempt - 1))
                    sleep = backoff + random.uniform(0, jitter)
                    # 简单日志
                    print(f'Retry attempt {attempt}/{total} after exception: {e}; sleeping {sleep:.2f}s')
                    time.sleep(sleep)
        return wrapper
    return decorator

# 使用示例（结合 requests）：
if __name__ == '__main__':
    import requests

    @retry(exceptions=(requests.exceptions.RequestException,), total=4, backoff_factor=0.3, jitter=0.2,
           allowed_result=lambda r: getattr(r, 'status_code', 200) < 500)
    def fetch(url):
        return requests.get(url, timeout=5)

    try:
        r = fetch('https://httpbin.org/status/500')
        print('最终状态码：', r.status_code)
    except Exception as e:
        print('请求失败：', e)


## 5. 使用 tenacity（更强大、可配置）
- 如果需要更复杂的策略（例如按异常类型分类重试、回调、停止条件、重试日志），建议使用 tenacity：
  https://github.com/jd/tenacity
```shell
pip install tenacity
```

In [ ]:
# tenacity 示例（仅展示用法，若未安装需先 pip install tenacity）
try:
    from tenacity import retry, stop_after_attempt, wait_exponential_jitter, retry_if_exception_type
    import requests

    @retry(stop=stop_after_attempt(5), wait=wait_exponential_jitter(initial=0.2, max=10),
           retry=retry_if_exception_type(requests.exceptions.RequestException))
    def fetch_tenacity(url):
        return requests.get(url, timeout=5)

    # 使用示例：
    # r = fetch_tenacity('https://httpbin.org/status/500')
except Exception as e:
    # 如果 tenacity 未安装，这里只是给出说明不抛错（笔记用途）
    pass


## 6. 常见陷阱与建议
- 不要在非幂等写操作上盲目重试：可能造成重复提交或副作用。
- 注意幂等性：PUT/DELETE 在多数情况下可视为幂等，但要结合业务判断。
- 合理设置超时：每次请求都应设 timeout，避免长时间阻塞重试循环。
- 退避参数要兼顾延迟与负载：指数退避 + 抖动是常见做法。
- 尊重目标服务器：遇到 429（限流）时适当延长重试间隔，并记录监控。
- 在分布式/并发场景下，避免所有客户端同步重试（使用抖动）。
- 日志要记录重试次数与失败原因，便于定位问题。

## 7. 小结与参考链接
- 优先使用 requests + urllib3 的 Retry + HTTPAdapter 能满足大部分需求。
- 对复杂需求使用 tenacity 或自定义装饰器。
- 关键点：设置合理的重试次数、超时、退避策略，并记录日志以避免对服务器造成额外压力。

参考：
- urllib3 Retry 文档: https://urllib3.readthedocs.io/en/latest/reference/urllib3.util.html#module-urllib3.util.retry
- tenacity: https://github.com/jd/tenacity
- requests 官方: https://docs.python-requests.org/